In [1]:

#    Multi-Disciplinary Design Optimisation of a Hydrogen Powered Commercial Aircraft (MDOHPCA)
#    this is the master file that runs each of the individual disciplines 
#    It sets up each of the analyses, then converts it into an openMDAO problem with a certain structure, then solves for minimum mass



In [2]:
import openmdao.api as om
import pandas as pd
import numpy as np

In [3]:
class MDOHPCA(om.Group):
    """
    Top level group containing the MDO.
    """           
    def setup(self):
        """
        set up each person's disciplines that can contain analyses
        """
        self.add_subsystem('aero', aero())
        self.add_subsystem('sys', sys())
        self.add_subsystem('stab', stab())
        self.add_subsystem('struct', struct())
        
        self.add_subsystem('mass', mass())
        
    def configure(self):
        #promote all variables (lazy option, they can be connected individually)
        self.promotes('sys',any=['*'])
        self.promotes('struct',any=['*'])
        
        self.promotes('mass',any=['*'])

In [4]:
class mass(om.ExplicitComponent):
    def setup(self):
        self.add_input('systems_mass')
        self.add_input('weight_structures')
        
        self.add_output('m')
        self.add_objective('m')
        
    def compute(self,inputs,outputs):
        outputs['m'] = inputs['systems_mass'] + inputs['weight_structures']

In [23]:
%run struct/structsetup.ipynb
#%run stab/stabsetup.ipynb
%run systems/systemssetup.ipynb
class aero(om.ExplicitComponent):
    pass
class stab(om.ExplicitComponent):
    pass

In [24]:
prob = om.Problem(model = MDOHPCA())
#prob.model.add_subsystem('aero', om.Group())

In [25]:
#setting inputs to reduce ambiguities where inputs differ in different disciplines
prob.model.set_input_defaults('CL', val = 0.6)
prob.model.set_input_defaults('sweep', val = 15)
prob.model.set_input_defaults('c', val = 4)
prob.model.set_input_defaults('b', val = 60)
prob.model.set_input_defaults('root_x', val = 10)

In [26]:
prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'
# prob.driver.options['maxiter'] = 100
prob.driver.options['tol'] = 1e-8
prob.driver.options["debug_print"] = ["nl_cons", "objs", "desvars"]

In [27]:
prob.setup()

In [28]:
#%run aero/aerosetup.ipynb

In [29]:
prob.run_model()

In [30]:
#get properties of specific constants in disciplines
prob.model.sys.options._dict['pp_mass']

{'val': 32816.0,
 'values': None,
 'types': None,
 'desc': 'people and cargo mass',
 'upper': None,
 'lower': None,
 'check_valid': None,
 'has_been_set': True,
 'allow_none': False,
 'recordable': True,
 'deprecation': None}

In [31]:
prob.model.list_outputs()

27 Explicit Output(s) in 'model'

varname                         val                 
------------------------------  --------------------
sys
  fuel_mass_calc
    fuel_mass                   [172168.2691172]    
  tanks
    tank1_mass_full             [122534.18152515]   
    tank1_mass_empty            [19233.22005483]    
    tank2_mass_full             [82151.13560505]    
    tank2_mass_empty            [13283.82795818]    
    CGtp                        [52.83836774]       
    tank1_x                     [21.95068906]       
    tank2_x                     [92.13594774]       
  pipes
    pipe_CG                     [39.52449579]       
    pipe_mass                   [112.59429651]      
  engine
    engine_mass                 [1054.575]          
    engine_x                    [14.82308546]       
    engine_y                    [18.]               
  actuator
    each_flap_actuator_mass     [2.55]              
    each_aileron_actuator_mass  [0.1275]            
    actu

[('sys.fuel_mass_calc.fuel_mass', {'val': array([172168.2691172])}),
 ('sys.tanks.tank1_mass_full', {'val': array([122534.18152515])}),
 ('sys.tanks.tank1_mass_empty', {'val': array([19233.22005483])}),
 ('sys.tanks.tank2_mass_full', {'val': array([82151.13560505])}),
 ('sys.tanks.tank2_mass_empty', {'val': array([13283.82795818])}),
 ('sys.tanks.CGtp', {'val': array([52.83836774])}),
 ('sys.tanks.tank1_x', {'val': array([21.95068906])}),
 ('sys.tanks.tank2_x', {'val': array([92.13594774])}),
 ('sys.pipes.pipe_CG', {'val': array([39.52449579])}),
 ('sys.pipes.pipe_mass', {'val': array([112.59429651])}),
 ('sys.engine.engine_mass', {'val': array([1054.575])}),
 ('sys.engine.engine_x', {'val': array([14.82308546])}),
 ('sys.engine.engine_y', {'val': array([18.])}),
 ('sys.actuator.each_flap_actuator_mass', {'val': array([2.55])}),
 ('sys.actuator.each_aileron_actuator_mass', {'val': array([0.1275])}),
 ('sys.actuator.actuators_CG_x', {'val': array([13.50885847])}),
 ('sys.systems_roundup